In [1]:
# !pip install openai python-aiconfig

In [2]:
# import openai
# import aiconfig
# from google.colab import userdata
# openai.api_key = userdata.get('OPENAI_API_KEY')


In [3]:
# Non-Colab
import json
import openai
openai.api_key = open("/home/jacobjensen/secrets/openai_api_key.txt", "r").read().strip()

## Function Calling

In [4]:
from pydantic import create_model
from typing import Callable, Dict, List
import inspect, json
from inspect import Parameter
def sums(a:int, b:int=1):
    "Adds a + b"
    return a + b
def schema(f):
    kw = {n:(o.annotation, ... if o.default==Parameter.empty else o.default)
          for n,o in inspect.signature(f).parameters.items()}
    s = create_model(f'Input for `{f.__name__}`', **kw).schema()
    return dict(name=f.__name__, description=f.__doc__, parameters=s)

In [5]:
def inject_function_call_spec(functions: List[Callable]):
    spec = json.load(open("function_calling_list_builder_aiconfig.json", 'r'))
    spec["prompts"][0]["metadata"]["model"]["settings"]["functions"] = [schema(f) for f in functions]
    return spec



In [6]:
async def initialize_list(items: List[str]) -> List[str]:
    """returns an initial list of items"""
    return items

async def add_to_list(li: List[str], items: List[str]) -> List[str]:
    """returns a list with items added, does not add duplicates"""
    return li + [item for item in items if item not in set(li)]

async def remove_from_list(li: List[str], items: List[str]) -> List[str]:
    """returns a list with items removed"""
    return [item for item in li if item not in set(items)]

# Note - this function isn't about calling an API or mutating state, but about enforcing an output format for assessments
async def assess_list(li: List[str], assessment: Dict[str, str]) -> List[str]: 
    """Accepts a dictionary of assessments of a list, keyed by the name of characteristic being assessed."""
    return assessment


In [7]:
import copy
import json
import uuid
from aiconfig import AIConfigRuntime
from aiconfig import PromptInput
from aiconfig.model_parser import InferenceOptions
from pprint import pprint


async def function_calling(function_list):
    function_call_spec = inject_function_call_spec(function_list)
    function_dict = {f.__name__: f for f in function_list}
    config = AIConfigRuntime.create(**function_call_spec)
    return config, function_dict

async def call_function(function_call, function_dict):
    args = function_call.get("arguments", None)
    args = json.loads(args) if args else None

    if not args:
        raise Exception("No arguments found")

    function = function_dict.get(function_call["name"], None)
    if not function:
        raise Exception(f"Function {function_call['name']} not found")
    else:
        return await function(**args)



/home/jacobjensen/.pyenv/versions/aiconfig/lib/python3.10/site-packages/pydantic/_internal/_fields.py:149: UserWarning: Field "model_parsers" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/home/jacobjensen/.pyenv/versions/aiconfig/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
build_list = await function_calling([initialize_list, add_to_list, remove_from_list, assess_list])

/tmp/ipykernel_485475/3314632738.py:11: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
  s = create_model(f'Input for `{f.__name__}`', **kw).schema()


In [9]:
config, function_dict = await function_calling([initialize_list, add_to_list, remove_from_list, assess_list])

/tmp/ipykernel_485475/3314632738.py:11: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
  s = create_model(f'Input for `{f.__name__}`', **kw).schema()


In [10]:
user_input = "I want to buy supplies for a 15 mile hike in hot weather."
li = []
promptToRun = "recommendList"
inference_options = InferenceOptions(stream=True)

async def run_chat(user_input, li, promptToRun=promptToRun, inference_options=inference_options):
    params = {
        "user_input": user_input,
        "list": li,
    }
    output = await config.run(promptToRun, params, inference_options)
    return output

while True:
    model_output = await run_chat(user_input, li, promptToRun, inference_options)

    output = model_output[0] if isinstance(model_output, list) else model_output
    print(output)

    if output.output_type == "error":
        raise ValueError(f"Error during inference: {output.ename}: {output.evalue}")

    message = output.data

    # If there is no function call, we're done and can exit this loop
    if not message.get("function_call", None):
        print("No function call found")
        break

    # If there is a function call, we generate a new message with the role 'function'.

    result = await call_function(message.get("function_call"), function_dict)
    print(result)

    new_message = {
        "role": "function",
        "name": message["function_call"]["name"],
        "content": json.dumps(result),
    }

    promptToRun = f"functionCallResult-{uuid.uuid4()}"

    existing_prompt = config.get_prompt("recommendList")

    new_prompt = copy.deepcopy(existing_prompt)
    new_prompt.name = promptToRun
    new_prompt.input = PromptInput(**new_message)
    new_prompt.outputs = []

    config.add_prompt(new_prompt.name, new_prompt)

    print(f"{new_message}\n")

output_type='execute_result' execution_count=0 data={'function_call': {'arguments': '{\n  "items": ["backpack", "water bottle", "hydration bladder", "snacks", "sunscreen", "hat", "sunglasses", "hiking shoes", "lightweight clothes", "first aid kit"]\n}', 'name': 'initialize_list'}, 'role': 'assistant'} mime_type=None metadata={'finish_reason': 'function_call'}
['backpack', 'water bottle', 'hydration bladder', 'snacks', 'sunscreen', 'hat', 'sunglasses', 'hiking shoes', 'lightweight clothes', 'first aid kit']
{'role': 'function', 'name': 'initialize_list', 'content': '["backpack", "water bottle", "hydration bladder", "snacks", "sunscreen", "hat", "sunglasses", "hiking shoes", "lightweight clothes", "first aid kit"]'}

Here is the updated list:

1. Backpack
2. Water bottle
3. Hydration bladder
4. Snacks
5. Sunscreen
6. Hat
7. Sunglasses
8. Hiking shoes
9. Lightweight clothes
10. First aid kitoutput_type='execute_result' execution_count=0 data={'content': 'Here is the updated list:\n\n1. Ba

In [11]:
current_args = json.loads(config.get_latest_output("recommendList").data["function_call"]["arguments"])
current_args
result

['backpack',
 'water bottle',
 'hydration bladder',
 'snacks',
 'sunscreen',
 'hat',
 'sunglasses',
 'hiking shoes',
 'lightweight clothes',
 'first aid kit']

In [12]:
params = {
        "user_input": "I want to buy supplies for a 15 mile hike in hot weather.",
        "list": "[]",
}

promptToRun = "recommendList"

inference_options = InferenceOptions(stream=True)

while True:
    model_output = await config.run(promptToRun, params, inference_options)

    output = model_output[0] if isinstance(model_output, list) else model_output

    if output.output_type == "error":
        raise ValueError(f"Error during inference: {output.ename}: {output.evalue}")

    message = output.data

    # If there is no function call, we're done and can exit this loop
    if not message.get("function_call", None):
        break

    # If there is a function call, we generate a new message with the role 'function'.

    result = await call_function(message.get("function_call"), function_dict)

    new_message = {
        "role": "function",
        "name": message["function_call"]["name"],
        "content": json.dumps(result),
    }

    promptToRun = f"functionCallResult-{uuid.uuid4()}"

    existing_prompt = config.get_prompt("recommendList")

    new_prompt = copy.deepcopy(existing_prompt)
    new_prompt.name = promptToRun
    new_prompt.input = PromptInput(**new_message)
    new_prompt.outputs = []

    config.add_prompt(new_prompt.name, new_prompt)

    print(f"{new_message}\n")

Here is the initial list of items for your 15 mile hike in hot weather:

- Backpack
- Water bottle
- Hydration bladder
- Snacks
- Sunscreen
- Hat
- Sunglasses
- Hiking shoes
- Lightweight clothes
- First aid kit

Is there anything else you would like to add to this list?{'role': 'function', 'name': 'initialize_list', 'content': '["backpack", "water bottle", "hydration bladder", "snacks", "sunscreen", "hat", "sunglasses", "hiking shoes", "lightweight clothes", "first aid kit"]'}

Here is the initial list of items for your 15 mile hike in hot weather:

- Backpack
- Water bottle
- Hydration bladder
- Snacks
- Sunscreen
- Hat
- Sunglasses
- Hiking shoes
- Lightweight clothes
- First aid kit

Is there anything else you would like to add to this list?

In [13]:
config.save("list_builder.json")